<a href="https://colab.research.google.com/github/mjiii25/prac_class/blob/main/LGBM%20%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [34]:
train = pd.read_csv('/content/drive/MyDrive/energy/new_test.csv의 사본')
test = pd.read_csv('/content/drive/MyDrive/energy/new_train.csv의 사본')
submission = pd.read_csv('/content/drive/MyDrive/energy/sample_submission.csv')

In [35]:
train.head()

,num,date_time,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,date,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,2020-08-25 00:00:00,27.800000,1.500000,74.000000,0.0,0.0,0,0,0,25,8,2020-08-25,1,0,78.602280,0,1,0,0,0,0.0,1.800000,30.027122,2
1,1,2020-08-25 01:00:00,27.633333,1.366667,75.333333,0.0,0.0,0,0,1,25,8,2020-08-25,1,0,78.519273,0,1,0,0,0,0.0,3.433333,29.859547,2
2,1,2020-08-25 02:00:00,27.466667,1.233333,76.666667,0.0,0.0,0,0,2,25,8,2020-08-25,1,0,78.431867,0,1,0,0,0,0.0,4.900000,29.694705,2
3,1,2020-08-25 03:00:00,27.300000,1.100000,78.000000,0.0,0.0,0,0,3,25,8,2020-08-25,1,0,78.340060,0,1,0,0,0,0.0,6.200000,29.533017,2
4,1,2020-08-25 04:00:00,26.900000,1.166667,79.666667,0.0,0.0,0,0,4,25,8,2020-08-25,1,0,77.912697,0,1,0,0,0,0.0,7.100000,29.116617,2


In [7]:
train.shape

(10080, 25)

In [37]:
#test.head()

In [10]:
test.shape

(122400, 26)

#### 군집화 결과 (수연님)

In [40]:
# 1번 군집 
list1 = [1, 31, 30, 60, 3, 5, 24,32, 33, 15]
# cl1_x = cl1.drop('temperature', axis = 1)
cl1 = train.drop(['date_time', 'date'], axis = 1)
cl1 = cl1[cl1['num'].isin(list1)==True]
#cl1 = train[train['num'].isin(list1)==True]
print(train.shape, cl1.shape)
print('1번 군집은 전체 데이터의 {0:.2f}'.format(cl1.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1680, 23)
1번 군집은 전체 데이터의 16.67 %를 차지함


In [13]:
# 2번 군집 
list1 = [2, 6, 7, 8, 13, 14, 16, 17, 18, 22, 23, 25, 26, 27, 37, 46, 47, 48, 52, 53, 54, 55, 56, 57 ,35, 38, 43, 44, 39, 45]
cl2 = train[train['num'].isin(list1)==True]
print(train.shape, cl2.shape)
print('2번 군집은 전체 데이터의 {0:.2f}'.format(cl2.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (5040, 25)
2번 군집은 전체 데이터의 50.00 %를 차지함


In [14]:
# 3번 군집
l2 = [19,20,21,29,36,40,41,49,50,58,59]
cl3 = train[train['num'].isin(l2)==True]
print(train.shape, cl3.shape)
print('3번 군집은 전체 데이터의 {0:.2f}'.format(cl3.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1848, 25)
3번 군집은 전체 데이터의 18.33 %를 차지함


In [15]:
# 4번 군집
l3 = [14,9,10,11,12,30,42]
cl4 = train[train['num'].isin(l3)==True]
print(train.shape, cl4.shape)
print('4번 군집은 전체 데이터의 {0:.2f}'.format(cl4.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1176, 25)
4번 군집은 전체 데이터의 11.67 %를 차지함


In [16]:
# 5번 군집
l4 = [5,21,28,32,51, 34]
cl5 = train[train['num'].isin(l4)==True]
print(train.shape, cl5.shape)
print('5번 군집은 전체 데이터의 {0:.2f}'.format(cl5.shape[0]/train.shape[0] *100 )+' %를 차지함')

(10080, 25) (1008, 25)
5번 군집은 전체 데이터의 10.00 %를 차지함


In [41]:
# 군집 5개 완성!
cl1.reset_index(inplace=True, drop=True)
#cl2.reset_index(inplace=True, drop=True)
#cl3.reset_index(inplace=True, drop=True)
#cl4.reset_index(inplace=True, drop=True)
#cl5.reset_index(inplace=True, drop=True)

In [42]:
# 기술통계량 확인
train.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.00000,10080.000000,10080.0,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.0
mean,30.500000,27.805813,2.440585,81.962500,2.191845,0.612917,0.683333,0.483333,11.50000,28.000000,8.0,3.000000,0.285714,79.447959,0.030853,0.579167,0.302679,0.079464,0.007837,1.049931,21.134927,30.073108,2.0
std,17.318961,2.275416,1.785098,11.467641,5.704649,0.867991,0.465199,0.499747,6.92253,2.000099,0.0,2.000099,0.451776,2.411122,0.172928,0.493717,0.459440,0.270475,0.088185,3.267854,21.159186,2.402481,0.0
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.00000,25.000000,8.0,0.000000,0.000000,71.780000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.600000,24.521506,2.0
25%,15.750000,26.133333,1.200000,74.666667,0.000000,0.000000,0.000000,0.000000,5.75000,26.000000,8.0,1.000000,0.000000,77.881755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.300000,28.329378,2.0
50%,30.500000,27.466667,2.100000,83.000000,0.000000,0.100000,1.000000,0.000000,11.50000,28.000000,8.0,3.000000,0.000000,79.214220,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.516667,29.646428,2.0
75%,45.250000,29.100000,3.175000,91.000000,1.500000,1.033333,1.000000,1.000000,17.25000,30.000000,8.0,5.000000,1.000000,81.019620,0.000000,1.000000,1.000000,0.000000,0.000000,0.000230,33.450000,31.386323,2.0
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.000000,1.000000,23.00000,31.000000,8.0,6.000000,1.000000,85.551900,1.000000,1.000000,1.000000,1.000000,1.000000,30.285714,88.600000,38.282359,2.0


In [43]:
cl1.describe()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.0,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.00000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.0
mean,23.400000,27.883095,2.973750,83.263095,1.869286,0.632321,0.800000,0.400000,11.500000,28.000000,8.0,3.000000,0.285714,79.748550,0.016071,0.562500,0.31369,0.095833,0.011905,1.240383,21.824544,30.125565,2.0
std,17.135191,2.228436,2.499206,11.672996,4.729929,0.898151,0.400119,0.490044,6.924248,2.000596,0.0,2.000596,0.451888,2.329356,0.125788,0.496226,0.46413,0.294450,0.108490,3.967847,21.159369,2.397801,0.0
min,1.000000,23.900000,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,8.0,0.000000,0.000000,73.364340,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-18.400000,25.767393,2.0
25%,5.000000,26.200000,1.366667,76.000000,0.000000,0.000000,1.000000,0.000000,5.750000,26.000000,8.0,1.000000,0.000000,78.172053,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,5.300000,28.326416,2.0
50%,27.000000,27.500000,2.400000,84.666667,0.000000,0.100000,1.000000,0.000000,11.500000,28.000000,8.0,3.000000,0.000000,79.456133,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,17.416667,29.673620,2.0
75%,32.000000,29.166667,3.766667,92.666667,1.100000,1.066667,1.000000,1.000000,17.250000,30.000000,8.0,5.000000,1.000000,81.158905,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,33.625000,31.465632,2.0
max,60.000000,34.600000,22.500000,100.000000,38.600000,3.000000,1.000000,1.000000,23.000000,31.000000,8.0,6.000000,1.000000,85.500240,1.000000,1.000000,1.00000,1.000000,1.000000,30.285714,80.100000,37.332811,2.0


In [20]:
#cl2.describe()

In [21]:
#cl3.describe()

In [22]:
#cl4.describe()

In [23]:
#cl5.describe()

#### LGBM 모델링

##### cluster1

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
SEED = 30

In [ ]:
def MSE(y, t) :
  return 0.5 * np.sum((y-t)**2)

In [46]:
cl1_x = cl1.drop('temperature', axis = 1)
cl1_y = cl1[['temperature']]

In [47]:
x_train, x_test, y_train, y_test = train_test_split(cl1_x, cl1_y, random_state=SEED)

In [53]:
lgbm = LGBMRegressor(random_state=SEED)
lgbm.fit(x_train, y_train)
lgbm_pred = lgbm.predict(x_test)
mse_eval('LGBM', lgbm_pred, y_test)

ValueError: ignored

-------------------------------------------------------
DACON 베이스라인

In [20]:
cross = KFold(n_splits = 5, shuffle = True, random_state = 42)
folds = []
for train_idx, valid_idx in cross.split(cl1_x, cl1_y):
  folds.append((train_idx, valid_idx))

In [21]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=cl1_x.iloc[train_idx, :]
    y_train=cl1_y.iloc[train_idx, :]
    X_valid=cl1_x.iloc[valid_idx, :]
    y_valid=cl1_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'================================================\n\n')

====================1=======================


ValueError: ignored